In [1]:
import pandas as pd
pd.set_option("display.max_columns", 150)
import sqlite3
import glob
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#importing the bike data from january seperatly because the columns are different than the rest of the year.
citibike_jan = pd.read_csv('../data/202101-citibike-tripdata_1.csv')
citibike_jan_2 = pd.read_csv('../data/202101-citibike-tripdata_2.csv')

In [3]:
# Importing all the files
csv_files = glob.glob('../data/2021/*.csv')

# List to collect DataFrames
dataframes = []


for csv_file in csv_files:
    df = pd.read_csv(csv_file, low_memory=False)
    dataframes.append(df)

citibike = pd.concat(dataframes, ignore_index=True)


citibike = citibike.reset_index(drop=True)   

In [4]:
#merging the two january files together. 
jan_citbike = pd.concat([citibike_jan, citibike_jan_2], ignore_index=True)

In [5]:
#removing spaces out of column names.
jan_citbike = jan_citbike.rename(columns = {'starttime': 'started_at', 'stoptime' : 'ended_at','start station id': 'start_station_id', 'start station name': 'start_station_name', 'start station latitude':'start_lat', 'start station longitude':'start_lng', 'end station id':'end_station_id', 'end station name':'end_station_name', 'end station latitude':'end_lat','end station longitude':'end_lng', 'usertype':'user_type', 'tripduration' : 'trip_duration'})

# #removing columns that are not also available in 2021 df
jan_citbike = jan_citbike.drop(columns = ['birth year', 'gender', 'bikeid'])

In [6]:
#selecting the columns I want in the citibike dataframe.
citibike = citibike[['started_at','ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual']]

citibike = citibike.rename(columns = {'member_casual':'user_type'})

In [7]:
#A function that changed the dtype of these columns to datetime.

def convert_datetime(df, columns):
    for column in columns:
        df[column] = pd.to_datetime(df[column])

columns = ['started_at', 'ended_at']

convert_datetime(citibike, columns)
convert_datetime(jan_citbike, columns)

In [8]:
#creating a column in 2021 dataframe that has the trip duration and then turning it into seconds.

citibike['trip_duration'] = citibike['ended_at'] - citibike['started_at']
citibike['trip_duration'] = citibike['trip_duration'].dt.total_seconds()

In [9]:
#changing the dtypes of columns, so that they the two dfs can merge.

jan_citbike['trip_duration'] = jan_citbike['trip_duration'].astype(float)
jan_citbike[['start_station_id', 'end_station_id']] = jan_citbike[['start_station_id', 'end_station_id']].astype(str)


In [10]:
#merging the citibike data from january to the rest of the months, finally!
citibike = pd.concat([jan_citbike, citibike], ignore_index=True)

In [11]:
# A function that will make new columns for the date, time, hour and day of the week each trip began and ended.

def datetime_date_time(df, columns):
    for column in columns:
        df[f'{column}_date'] = pd.to_datetime(df[column]).dt.date
        df[f'{column}_time'] = pd.to_datetime(df[column]).dt.time
        df[f'{column}_hour'] = pd.to_datetime(df[column]).dt.hour
        df[f'{column}_day_of_week'] = pd.to_datetime(df[column]).dt.weekday
        df[f'{column}_day_of_year']= pd.to_datetime(df[column]).dt.dayofyear
        
columns = ['started_at', 'ended_at']

datetime_date_time(citibike, columns)

In [12]:
#using the convert_datetime function to convert the date column to a datetime dtype.

columns = ['started_at_date', 'ended_at_date']

convert_datetime(citibike, columns)

In [13]:
citibike = citibike.sort_values('started_at_date')
citibike

,trip_duration,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,user_type,started_at_date,started_at_time,started_at_hour,started_at_day_of_week,started_at_day_of_year,ended_at_date,ended_at_time,ended_at_hour,ended_at_day_of_week,ended_at_day_of_year
0,2513.0,2021-01-01 00:00:11.902,2021-01-01 00:42:05.226,3581,Underhill Ave & Lincoln Pl,40.674012,-73.967146,3581,Underhill Ave & Lincoln Pl,40.674012,-73.967146,Customer,2021-01-01,00:00:11.902000,0,4,1,2021-01-01,00:42:05.226000,0,4,1
12530,1756.0,2021-01-01 14:48:54.345,2021-01-01 15:18:10.688,501,FDR Drive & E 35 St,40.744219,-73.971212,127,Barrow St & Hudson St,40.731724,-74.006744,Subscriber,2021-01-01,14:48:54.345000,14,4,1,2021-01-01,15:18:10.688000,15,4,1
12529,2462.0,2021-01-01 14:48:51.248,2021-01-01 15:29:53.461,3521,Lenox Ave & W 111 St,40.798786,-73.952300,3551,3 Ave & E 112 St,40.795508,-73.941606,Subscriber,2021-01-01,14:48:51.248000,14,4,1,2021-01-01,15:29:53.461000,15,4,1
12528,1601.0,2021-01-01 14:48:49.495,2021-01-01 15:15:30.944,412,Forsyth St & Canal St,40.715815,-73.994224,412,Forsyth St & Canal St,40.715815,-73.994224,Subscriber,2021-01-01,14:48:49.495000,14,4,1,2021-01-01,15:15:30.944000,15,4,1
12527,1181.0,2021-01-01 14:48:48.391,2021-01-01 15:08:30.341,3139,E 72 St & Park Ave,40.771183,-73.964094,517,Pershing Square South,40.751581,-73.977910,Customer,2021-01-01,14:48:48.391000,14,4,1,2021-01-01,15:08:30.341000,15,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10819735,1624.0,2021-12-31 15:41:32.000,2021-12-31 16:08:36.000,7520.07,Columbus Ave & W 95 St,40.791956,-73.968087,6551.02,W 44 St & 5 Ave,40.755003,-73.980144,member,2021-12-31,15:41:32,15,4,365,2021-12-31,16:08:36,16,4,365
10653428,688.0,2021-12-31 13:30:50.000,2021-12-31 13:42:18.000,5561.06,Lafayette St & Jersey St,40.724561,-73.995653,5914.08,Greenwich Ave & Charles St,40.735238,-74.000271,casual,2021-12-31,13:30:50,13,4,365,2021-12-31,13:42:18,13,4,365
11387493,473.0,2021-12-31 14:03:20.000,2021-12-31 14:11:13.000,4225.14,Carroll St & Smith St,40.680611,-73.994758,4565.04,Schermerhorn St & Court St,40.691029,-73.991834,member,2021-12-31,14:03:20,14,4,365,2021-12-31,14:11:13,14,4,365
11909001,1686.0,2021-12-31 14:44:07.000,2021-12-31 15:12:13.000,3492.04,34 St & 4 Ave,40.655278,-74.003101,3579.04,Windsor Pl & Howard Pl,40.659491,-73.980139,casual,2021-12-31,14:44:07,14,4,365,2021-12-31,15:12:13,15,4,365


In [14]:
#A function that will create a column for the month of the trip.

def datetime_month(df, columns):
    for column in columns:
        df[f'{column}_month'] = pd.DatetimeIndex(df[column]).month
        
columns = ['started_at', 'ended_at']

datetime_month(citibike, columns)

In [15]:
#https://stackoverflow.com/questions/29688899/pandas-checking-if-a-date-is-a-holiday-and-assigning-boolean-value

#A datframe that will define if a date was a holiday.

dr = pd.date_range(start='2021-01-01', end='2021-12-31')
df = pd.DataFrame()
df['date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

df['holiday'] = df['date'].isin(holidays)


**Using the weather**

In [16]:
#importing the weather file.
cols = ['PRCP', 'SNOW', 'TMAX', 'TMIN', 'DATE', 'RHAV', 'AWND', 'RHMN', 'RHMX']
weather = pd.read_csv('../data/USW00094728.csv',usecols = cols, low_memory = False)

In [17]:
#changing all the column titles to lowercase.
weather.columns = map(str.lower, weather.columns)

#changing the DATE column to a datetime datatype.
weather['date'] = pd.to_datetime(weather['date'])

#choosing the years I want data in.
weather = weather[weather['date'].dt.year.isin([2021])].reset_index(drop = True)

In [18]:
day_trips = citibike.groupby('started_at_date').size().reset_index(name = 'num_of_trips')
day_trips = day_trips.rename(columns = {'started_at_date' : 'date'})

In [19]:
day_trips = day_trips.merge(df, left_on = 'date', right_on = 'date', how = 'left')
day_trips = day_trips.merge(weather, left_on = 'date', right_on = 'date', how = 'left')

In [20]:
#Changing the temperature to fahrenheit.
#the original temperature was the celsius temp x 10. 
def to_fahrenheit(df, columns):
    for column in columns:
        df[f'{column}_fahrenheit'] = (df[column]/10)*(9/5)+32
columns = ['tmax', 'tmin']

to_fahrenheit(day_trips, columns)

In [21]:
#Droping the original temp columns and renaming the farenheit temp columns.

day_trips = day_trips.drop(columns = ['tmax','tmin'])
day_trips = day_trips.rename(columns = {'tmax_fahrenheit' : 'tmax', 'tmin_fahrenheit' : 'tmin'})

day_trips['tavg'] = (day_trips['tmax'] + day_trips['tmin'])/2

In [22]:
median_trips = round(citibike.groupby('started_at_date')['trip_duration'].median(), 2) #df that has the date and medians trip duration.
day_trips = day_trips.merge(median_trips, left_on = 'date', right_on = 'started_at_date', how = 'left') #merging the df to day_trips.
day_trips = day_trips.rename(columns = {'trip_duration' : 'median_trip_duration'}) #changing the name of the column.

In [23]:
mean_trips = round(citibike.groupby('started_at_date')['trip_duration'].mean(), 2)
day_trips = day_trips.merge(mean_trips, left_on = 'date', right_on = 'started_at_date', how = 'left') #merging the df to day_trips.
day_trips = day_trips.rename(columns = {'trip_duration' : 'mean_trip_duration'}) #changing the name of the column.

In [24]:
#Adding a day of the week column.
day_trips['day_of_week'] = pd.to_datetime(day_trips['date']).dt.weekday

In [25]:
day_trips

,date,num_of_trips,holiday,prcp,snow,awnd,rhav,rhmn,rhmx,tmax,tmin,tavg,median_trip_duration,mean_trip_duration,day_of_week
0,2021-01-01,18790,True,157,0.0,25.0,69.0,55.0,93.0,39.92,33.08,36.50,665.0,1273.26,4
1,2021-01-02,41195,False,13,0.0,30.0,74.0,54.0,97.0,51.08,35.96,43.52,783.0,1208.44,5
2,2021-01-03,14578,False,56,0.0,41.0,83.0,70.0,93.0,37.94,33.98,35.96,508.0,1028.81,6
3,2021-01-04,36745,False,5,0.0,20.0,77.0,67.0,89.0,42.98,35.06,39.02,569.0,888.63,0
4,2021-01-05,38525,False,0,0.0,22.0,69.0,60.0,79.0,42.08,35.96,39.02,570.0,877.76,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2021-12-27,31583,False,23,0.0,20.0,63.0,51.0,93.0,39.02,33.98,36.50,511.0,1452.87,0
360,2021-12-28,40728,False,13,0.0,21.0,72.0,51.0,96.0,46.94,35.96,41.45,574.0,1614.75,1
361,2021-12-29,39656,False,36,0.0,26.0,90.0,85.0,93.0,44.06,41.00,42.53,558.0,1450.41,2
362,2021-12-30,46925,False,13,0.0,11.0,87.0,80.0,97.0,48.92,42.98,45.95,587.0,1507.32,3


In [26]:
# #saving df to csv
# day_trips.to_csv('../data/day_trips_2021')

***Answering data questions***

*What time are the bikes most frequently used during the day?*

In [ ]:
#The hour the most amount of trips occur.
def most_hour (df, year):
    hour_count = df.groupby('started_at_hour')['started_at_hour'].count().sort_values(ascending=False)
    most_popular = hour_count.index[0]
    number_of_rides = hour_count.iloc[0]
    print(f"The hour with the most amount of trips in {year} was {most_popular} with {number_of_rides} rides.")

most_hour(citibike, 2021)

*What time are the bikes used the least during the day?*

In [ ]:
#The hour the least amount of trips occur.
def least_hour (df, year):
    hour_count = df.groupby('started_at_hour')['started_at_hour'].count().sort_values(ascending=True)
    least_popular = hour_count.index[0]
    number_of_rides = hour_count.iloc[0]
    print(f"The hour with the least amount of trips in {year} was {least_popular} with {number_of_rides} rides.")

least_hour(citibike, 2021)

*Which stations are the most frequent for starting & ending a trip?*

In [ ]:
#The station where the most trips begin
def popular_starting_station (df, year):
    station_count = df.groupby('start_station_name')['start_station_name'].count().sort_values(ascending=False)
    most_popular = station_count.index[0]
    number_of_rides = station_count.iloc[0]
    print(f"The station where most trips began in {year} was {most_popular} with {number_of_rides} rides.")

popular_starting_station(citibike, 2021)

In [ ]:
#The station where the most trips ended.
def popular_ending_station (df, year):
    station_count = df.groupby('end_station_name')['end_station_name'].count().sort_values(ascending=False)
    most_popular = station_count.index[0]
    number_of_rides = station_count.iloc[0]
    print(f"The station where most trips ended in {year} was {most_popular} with {number_of_rides} rides.")

popular_ending_station(citibike, 2021)

**What is the average trip length? Does it change depending on day or time of day?**

In [ ]:
#The average bike trip.
def avg_bike_trip (df, year):
    for column in columns:
        avg_trip_len = round(df[column].mean())
        print(f"The average length of a bike trip in {year} is {avg_trip_len} seconds")
        
columns = ['trip_duration']
avg_bike_trip(citibike, 2021)

In [ ]:
#The hour with the longest bike trip.
def avg_bike_trip_by_hour (df, year):
    for column in columns:
        avg_trip_len = round(df.groupby('started_at_hour')['trip_duration'].mean().sort_values(ascending = False))
        hour_with_longest_trips = avg_trip_len.index[0]
        avg_seconds = avg_trip_len.iloc[0]
        print(f"In {year}, the hour that has the longest bike trip on average is hour {hour_with_longest_trips}, which lasts about {avg_seconds} seconds")
        
avg_bike_trip_by_hour(citibike, 2021)

In [ ]:
#The hour with the shortest bike trip.
def avg_bike_trip_by_hour_least (df, year):
    for column in columns:
        avg_trip_len = round(df.groupby('started_at_hour')['trip_duration'].mean().sort_values(ascending = True))
        hour_with_shortest_trips = avg_trip_len.index[0]
        avg_seconds = avg_trip_len.iloc[0]
        print(f"In {year}, the hour that has the shortest bike trip on average is hour {hour_with_shortest_trips}, which lasts about {avg_seconds} seconds")
        
avg_bike_trip_by_hour_least(citibike, 2021)

In [ ]:
#The day of year with the longest trips.
def avg_bike_trip_by_day_of_year (df, year):
    for column in columns:
        avg_trip_len = round(df.groupby('started_at_day_of_year')['trip_duration'].mean().sort_values(ascending = False))
        day_of_year_with_longest_trips = avg_trip_len.index[0]
        avg_seconds = avg_trip_len.iloc[0]
        print(f"In {year}, the day of year that has the longest bike trip on average is day {day_of_year_with_longest_trips}, which lasts about {avg_seconds} seconds")
        
avg_bike_trip_by_day_of_year(citibike, 2021)

In [ ]:
#The day of year with the shortest trips.
def avg_bike_trip_by_day_of_year_shortest (df, year):
    for column in columns:
        avg_trip_len = round(df.groupby('started_at_day_of_year')['trip_duration'].mean().sort_values(ascending = True))
        day_of_year_with_shortest_trips = avg_trip_len.index[0]
        avg_seconds = avg_trip_len.iloc[0]
        print(f"In {year}, the day of year that has the shortest bike trip on average is day {day_of_year_with_shortest_trips}, which lasts about {avg_seconds} seconds")
        
avg_bike_trip_by_day_of_year_shortest(citibike, 2021)